In [1]:
library(BSgenome.Hsapiens.UCSC.hg38)
library(data.table)
for (i in names(Hsapiens)[1:24]){ # load all chromosome map files
        load(paste0('./data/large_files/',i,'.rda'))
}
load('./data/exons.rda')
weights <- read.csv('./data/snap_velcro_weights.csv')
weights <- weights[7,2:5]

Loading required package: BSgenome
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colMeans,
    colnames, colSums, dirname, do.call, duplicated, eval, evalq,
    Filter, Find, get, grep, grepl, intersect, is.unsorted, lapply,
    lengths, Map, mapply, match, mget, order, paste, pmax, pmax.int,
    pmin, pmin.int, Position, rank, rbind, Reduce, rowMeans, rownames,
    rowSums, sapply, setdiff, sort, table, tapply, union, unique,
    unsplit, which, which.max, which.min

Loading required package: S4

In [2]:
# Take the union of overlapping exonic ranges, and omit parent gene data
exons <- GRanges(exons$chrom,IRanges(exons$start,exons$end))
exons <- reduce(exons)
exons <- data.table(chrom=as.vector(seqnames(exons)),start=start(exons),end=end(exons))
head(exons)

chrom,start,end
1,65419,65433
1,65520,65573
1,69037,71585
1,450703,451697
1,685679,686673
1,923928,924948


#### In the cell below, we count the number of L1 target sites of each Snap-Velcro type in exons and non-exonic regions

In [3]:
counts <-  array(0,dim=c(24,8)) # Allocate matrix for counts
exon_lens <- rep(0,24)

#--- Loop through chromosome names
j<-1 # chromosome counter
for (i in names(Hsapiens)[1:24]){


        # Data objects containing indices of S-V sites for each category are labeled with the chromosome name.
        # Here we copy the data objects to a set of variables with generic names which can be used consistently 
        # in the loop. 
        map<-get(paste0(i,'Map')) # Get the site map data for the current chrom
        ict<-    map[[2]]
        icl<-    map[[3]]
        iot<-    map[[4]]
        iol<-    map[[5]]
        insites<-map[[1]] 
    
        i <- strsplit(i,"chr")[[1]][2] # Remove "chr" from chromosome name

################################################
        # count in exons for current chromosome (columns 1-4)
        ann_i <- exons[exons$chrom == i,] 
    
        sen        <-inrange(insites[ict[!is.na(ict[,1]),1],1],ann_i$start,ann_i$end) # Check if any Closed-Tight category sites are within the start-end range of ann_i
        antisen    <-inrange(insites[ict[!is.na(ict[,2]),2],2],ann_i$start,ann_i$end) 
        counts[j,1]<-length(which(sen)) + length(which(antisen))                      # Fill an element of the counts table with the count
        
        sen        <-inrange(insites[icl[!is.na(icl[,1]),1],1],ann_i$start,ann_i$end) 
        antisen    <-inrange(insites[icl[!is.na(icl[,2]),2],2],ann_i$start,ann_i$end)
        counts[j,2]<-length(which(sen)) + length(which(antisen))
        
        sen        <-inrange(insites[iot[!is.na(iot[,1]),1],1],ann_i$start,ann_i$end) 
        antisen    <-inrange(insites[iot[!is.na(iot[,2]),2],2],ann_i$start,ann_i$end)
        counts[j,3]<-length(which(sen)) + length(which(antisen))
        
        sen        <-inrange(insites[iol[!is.na(iol[,1]),1],1],ann_i$start,ann_i$end) 
        antisen    <-inrange(insites[iol[!is.na(iol[,2]),2],2],ann_i$start,ann_i$end)
        counts[j,4]<-length(which(sen)) + length(which(antisen))  
    
################################################
        # count in non-exons for current chromosome (columns 5-8)
        
        counts[j,5]   <- length(which(!is.na(as.vector(ict)))) - counts[j,1]
        counts[j,6]  <- length(which(!is.na(as.vector(icl)))) - counts[j,2]
        counts[j,7]  <- length(which(!is.na(as.vector(iot)))) - counts[j,3]
        counts[j,8]  <- length(which(!is.na(as.vector(iol)))) - counts[j,4]
    
        exon_lens[j] <- sum(ann_i$end-ann_i$start)
    
        j<-j+1
        rm('sen','antisen')
}

In [4]:
colnames(counts) <- c('ct_exon','cl_exon','ot_exon','ol_exon','ct_nonexon','cl_nonexon','ot_nonexon','ol_nonexon')
rownames(counts) <- names(Hsapiens)[1:24]
# counts

Here we account for male and female diploid genomes

#### Male

In [5]:
# Male genome - double the count for chromosomes 1:22
counts_dipl_m <- counts
counts_dipl_m[1:22,]<-counts_dipl_m[1:22,]*2
# counts_dipl_m

#### Female

In [6]:
# Female genome - double the counts for chroms. 1:23; count for chrom. Y is set to 0
counts_dipl_f <- counts
counts_dipl_f[1:23,]<-counts_dipl_f[1:23,]*2
counts_dipl_f[24,]<-counts_dipl_f[24,]*0
# counts_dipl_f

## Calculating probabilities of ENd (endonuclease-dependent) exonic vs non-exonic insertion

#### Male

In [7]:
counts_ovchr_m <- colSums(counts_dipl_m)
probs_ENd_m <- rep(0,2)
for (i in 1:2){
    probs_ENd_m[i] = weights$closed_tight*counts_ovchr_m[(i-1)*4+1] + 
                   weights$closed_loose*counts_ovchr_m[(i-1)*4+2] + 
                   weights$open_tight*counts_ovchr_m[(i-1)*4+3] + 
                   weights$open_loose*counts_ovchr_m[(i-1)*4+4]
}
probs_ENd_m <- probs_ENd_m/sum(probs_ENd_m)
probs_ENd_m

[1] 0.02372577 0.97627423

#### Female

In [8]:
counts_ovchr_f <- colSums(counts_dipl_f)
probs_ENd_f <- rep(0,2)
for (i in 1:2){
    probs_ENd_f[i] = weights$closed_tight*counts_ovchr_f[(i-1)*4+1] + 
                   weights$closed_loose*counts_ovchr_f[(i-1)*4+2] + 
                   weights$open_tight*counts_ovchr_f[(i-1)*4+3] + 
                   weights$open_loose*counts_ovchr_f[(i-1)*4+4]
}
probs_ENd_f <- probs_ENd_f/sum(probs_ENd_f)
probs_ENd_f

[1] 0.02357664 0.97642336

## Calculating probabilities of ENi (endonuclease-independent) exonic vs non-exonic insertion

In [9]:
chrlens <- rep(0,24)
for (i in 1:24){
    chrlens[i] <- length(Hsapiens[[i]])
}
sum(chrlens)

[1] 3088269832

#### Male

In [10]:
probs_ENi_m <- rep(0,2)
probs_ENi_m[1] <- (sum(exon_lens[1:22]*2)+sum(exon_lens[23:24]))/(sum(chrlens[1:22]*2)+sum(chrlens[23:24]))
probs_ENi_m[2] <- 1-probs_ENi_m[1]
probs_ENi_m

[1] 0.03276097 0.96723903

#### Female

In [11]:
probs_ENi_f <- rep(0,2)
probs_ENi_f[1] <- sum(exon_lens[1:23]*2)/sum(chrlens[1:23]*2)
probs_ENi_f[2] <- 1-probs_ENi_f[1]
probs_ENi_f

[1] 0.0327668 0.9672332

## Calculating final probabilities of exonic vs non-exonic insertion

#### Male

In [12]:
pd_exvsnon_m <- (.9*probs_ENd_m)+(0.1*probs_ENi_m) # Assuming 0.9 probability that insertion will be ENd and 0.1 for ENi
pd_exvsnon_m <- pd_exvsnon_m/sum(pd_exvsnon_m)
pd_exvsnon_m

[1] 0.02462929 0.97537071

#### Female

In [13]:
pd_exvsnon_f <- (.9*probs_ENd_f)+(0.1*probs_ENi_f) # Assuming 0.9 probability that insertion will be ENd and 0.1 for ENi
pd_exvsnon_f <- pd_exvsnon_f/sum(pd_exvsnon_f)
pd_exvsnon_f

[1] 0.02449566 0.97550434

In [14]:
lens <- cbind(exon_lens, chrlens)
lens

exon_lens,chrlens
9914687,248956422
7203029,242193529
5937840,198295559
4055667,190214555
4778798,181538259
4799952,170805979
4696552,159345973
3495005,145138636
3762545,138394717
3707409,133797422


In [15]:
save(lens,counts_dipl_m,counts_dipl_f,pd_exvsnon_m,pd_exvsnon_f,file='./data/exonicvsnon_counts.rda')